In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #~180 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossing')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
# print(len(nodes))
edges = G.edges
# print(len(max(nx.connected_components(G), key=len)))

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
#Show the map as nodes and edges

from shapely.geometry import Point
# start_node = gpd.GeoDataFrame({'geometry': [Point(122245.37633330293,
#                                                   486126.8581684635)]})
# end_node = gpd.GeoDataFrame({'geometry': [Point(122247.04588395767,
#                                                   486167.91526857053)]})
# fig, ax = plt.subplots(figsize=(14,14), dpi=600)
# # All nodes and edges
# nx.draw(G, {n:[n[0], n[1]] for n in list(G.nodes)}, ax=ax, node_size=3)
# # Start & end node
# start_node.plot(ax=ax, color='red')
# end_node.plot(ax=ax, color='purple')
# plt.show()

In [5]:
# print(nodes)

In [6]:
#Pick random ones or pick manually that make sense - to experiment
#Smaller map:
S = (122245.37633330293, 486126.8581684635)
T = (122246.77932030056, 486223.5791244763)
#Small map:
# S = (120558.58272730978, 486088.5913559315)
# T = (120683.06067979027, 485777.31229122455)

In [7]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [71]:
P = [] #Pareto set
p = [] #paths computed by Dijkstra's algorithm
val_vector_p = [] #value vectors w.r.t. p, i.e., v^{p_1}, v^{p_2}

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra's algorithm
    P.append(p)

    val_obj1 = nx.path_weight(G, path=p, weight='length') #Returns total cost associated with the path and weight. In other words, it returns the value of the path.
    val_obj2 = nx.path_weight(G, path=p, weight='crossing')
    val_vector_p.append(np.array([val_obj1, val_obj2]))

In [72]:
val_vector_p

[array([165.21,   2.  ]), array([227.59,   2.  ])]

In [10]:
#Candidate Targets, i.e., the most optimistic points
C = [min(val_vector_p[0][0], val_vector_p[1][0]), min(val_vector_p[0][1], val_vector_p[1][1])]
C = [np.array(C)]
C

[array([165.21,   2.  ])]

In [65]:
# The most pessimistic points form the upper bounds
U = [max(val_vector_p[0][0], val_vector_p[1][0]), max(val_vector_p[0][1], val_vector_p[1][1])]
U

[227.59000000000003, 2.0]

In [11]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [12]:
add_noise = True
ground_utility = user_preference.get_preference(val_vector_p, add_noise=add_noise) #This is the ground-truth utility, i.e., the true utility
print(ground_utility)

[4.5592     4.40645459]


In [13]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)

comparisons.add_single_comparison(val_vector_p[np.argmax(ground_utility)], val_vector_p[np.argmin(ground_utility)]) #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

[[165.21   2.  ]
 [227.59   2.  ]]


In [14]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
u_v, _ = gp.get_predictive_params(val_vector_p, True) #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
print(u_v)

[ 0.02560488 -0.02560488]


In [15]:
p_star_index = np.argmax(u_v)
p_star_index

0

In [16]:
p_star = P[p_star_index]
p_star

[(122245.37633330293, 486126.8581684635),
 (122254.86602688645, 486129.80052856216),
 (122264.35426393585, 486132.74411788705),
 (122273.84081337851, 486135.69101398275),
 (122283.19823912054, 486138.9678026403),
 (122284.1816391946, 486139.19731384714),
 (122290.12705889999, 486148.4950394649),
 (122296.3898333376, 486151.2069021054),
 (122293.35529070727, 486160.70100324217),
 (122290.30564961619, 486170.1963395323),
 (122289.24584929895, 486173.25169782597),
 (122284.2947638598, 486179.51092348614),
 (122286.01308887315, 486183.8814735307),
 (122282.69556966702, 486193.2948693236),
 (122279.36739295325, 486202.70485648955),
 (122276.034562429, 486212.11332275654),
 (122274.00277185065, 486217.2508821529),
 (122271.878266905, 486221.2167929504),
 (122262.38569380266, 486218.13414513733),
 (122254.26477155345, 486215.7695028797),
 (122253.09793657108, 486219.18429932056),
 (122246.77932030056, 486223.5791244763)]

In [17]:
input_domain = np.array(C) #set of candidate targets
print(input_domain)

[[165.21   2.  ]]


In [18]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=input_domain, query_type='ranking', seed=123, acquisition_type='expected improvement')

In [19]:
# TODO: The next code cells are in a while-loop
# while C:
#     expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
#     t_index = np.argmax(expected_improvement)
#     t = C[t_index]
#     C.remove(t)
#
# # t_index
# print(t)

In [20]:
expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
print(expected_improvement)

[0.14548911]


In [21]:
t_index = np.argmax(expected_improvement)
t_index

0

In [22]:
t = input_domain[t_index]
print(t)

[165.21   2.  ]


In [23]:
# Remove t from C
C = np.delete(C, np.where(np.all(C == t)))
print(C)

[2.]


In [24]:
v_n = {}
threshold=1e-8
max_iter=1000
obj = 'length'
next_node = {}
edge_cost = []

for n in G:
    v_n[n] = np.inf
    if n == T:
        v_n[n] = 0

for i in range(max_iter): #or until convergence

    converged = True

    # print("iteration:", i)
    for e in G.edges(data=True):
        n1, n2 = e[0], e[1]
        # if v_n[n1] != np.inf or v_n[n2] != np.inf:
        #     print(e[0], e[1], ":", v_n[n1], v_n[n2])

        # print(n2 in G)
        cost = e[2][obj]
        edge_cost.append(cost)
        # print(cost)

            # print(e)
        result1 = min(cost + v_n[n2], v_n[n1])
        result2 = min(cost + v_n[n1], v_n[n2])
            # print("result1:", result1, "sum1:", cost + v_n[n2])
            # print("result2", result2, "sum2:", cost + v_n[n1])
        if v_n[n1] != result1 or v_n[n2] != result2:
            converged = False

        if v_n[n1] != result1:
            next_node[n1] = n2

        if v_n[n2] != result2:
            next_node[n2] = n1

        v_n[n1] = result1
        v_n[n2] = result2

        # if n1 == T or n2==T:
        #     # print(result1, result2)
        #     print(v_n[n1], v_n[n2])


    # converged = all(n in v_n_copy and abs(v_n[n] - v_n_copy[n]) < threshold for n in v_n)
    if converged:
        break


        # if v_n[n1] != np.inf:
        #     print(n1,v_n[n1])
        # if v_n[n2] != np.inf:
        #     print(n2,v_n[n2])

print(v_n)
# print(next_node)

# S = (122245.37633330293, 486126.8581684635)
# T = (122246.77932030056, 486223.5791244763)

{(122245.37633330293, 486126.8581684635): 165.21, (122254.86602688645, 486129.80052856216): 155.22, (122264.35426393585, 486132.74411788705): 145.23, (122273.84081337851, 486135.69101398275): 135.23999999999998, (122283.19823912054, 486138.9678026403): 125.24999999999999, (122284.1816391946, 486139.19731384714): 124.23999999999998, (122287.28805009159, 486137.97386694513): 127.58999999999997, (122291.40829399273, 486140.643590483): 132.52999999999997, (122289.53424561612, 486128.31132216856): 137.57999999999998, (122297.7854183222, 486139.47700177913): 139.09999999999997, (122301.24255780602, 486130.1701925009): 149.08999999999997, (122304.47424118801, 486120.76208520425): 159.07999999999998, (122256.54763350038, 486170.91398380254): 104.02, (122247.04588395767, 486167.91526857053): 114.00999999999999, (122237.50744934028, 486164.9669819529): 123.99999999999999, (122227.969152229, 486162.0189639581): 133.98999999999998, (122218.43104837791, 486159.0709819753): 143.98, (122289.245849298

In [25]:
import single_vi_iter
lower_bounds = []
next_nodes = []

lower_length, cost_length, next_node_length = single_vi_iter.single_value_iter(G, T, 'length')
lower_crossing, cost_crossing, next_node_crossing = single_vi_iter.single_value_iter(G, T, 'crossing')
next_nodes.append(np.array([next_node_length, next_node_crossing]))
# print(cost_length)
# print("NEXT:", next_node_crossing)

lower_bounds.append(np.array([lower_length, lower_crossing]))
lower_bounds

#crossing: (122246.10058319086, 486224.512771215): (122246.77932030056, 486223.5791244763)

[array([{(122245.37633330293, 486126.8581684635): 165.21, (122254.86602688645, 486129.80052856216): 155.22, (122264.35426393585, 486132.74411788705): 145.23, (122273.84081337851, 486135.69101398275): 135.23999999999998, (122283.19823912054, 486138.9678026403): 125.24999999999999, (122284.1816391946, 486139.19731384714): 124.23999999999998, (122287.28805009159, 486137.97386694513): 127.58999999999997, (122291.40829399273, 486140.643590483): 132.52999999999997, (122289.53424561612, 486128.31132216856): 137.57999999999998, (122297.7854183222, 486139.47700177913): 139.09999999999997, (122301.24255780602, 486130.1701925009): 149.08999999999997, (122304.47424118801, 486120.76208520425): 159.07999999999998, (122256.54763350038, 486170.91398380254): 104.02, (122247.04588395767, 486167.91526857053): 114.00999999999999, (122237.50744934028, 486164.9669819529): 123.99999999999999, (122227.969152229, 486162.0189639581): 133.98999999999998, (122218.43104837791, 486159.0709819753): 143.98, (122289.2

In [26]:
def cost_to(current_node, obj): # Cost from S to current_node for 1 objective
    total_cost = 0

    for e in G.edges(data=True):
        # print(e[0])
        edge_cost = e[2][obj]

        if e[0] == current_node or e[1] == current_node:
            break

        total_cost += edge_cost
    return total_cost

cost_both_obj = []
current_n = (122284.1816391946, 486139.19731384714)
obj1 = cost_to(current_n, 'length')
obj2 = cost_to(current_n, 'crossing')
cost_both_obj.append(np.array([obj1, obj2]))
print(cost_both_obj)

[array([39.96,  0.  ])]


In [91]:
i = 0  # track iterations of the algorithm
max_iter = 1000

stack = [(S, [S])]  # (starting node, cost, path), where cost is from previous_state to S, path is from S to current_state (i.e., S)

r1 = 0 # Result from objective length
r2 = 0 # Result from objective crossing
result = [] # The new lower bound

min_distance = np.inf
closest_node = None
val_vector_path = []

while stack:
    current_node, path = stack.pop()  # cost=total cost up to the current_node

    if current_node == T:
        # print(f"Path {path} with cost {cost}")
        print(f"Path {path} with cost {val_vector_path}")
        #TODO: How to calculate the cost of this path?

    # for neighbor in [next_node_length, next_node_crossing]:
    #     cost_len = cost_to(neighbor, 'length')
    #     cost_cross = cost_to(neighbor, 'crossing')
    #     cost_obj.append(np.array([cost_len, cost_cross]))
    #
    # for neighbor in next_nodes:
    #     result = cost_obj + lower_bounds
    # print(result)

    for neighbor in next_node_length:
        cost_len = cost_to(neighbor, 'length')
        r1 = cost_len + lower_length[neighbor]
        # print("cost:", cost_len)
        # print("bound:", lower_length[neighbor])
        # print("R1:", r1)
    for neighbor in next_node_crossing:
        cost_cross = cost_to(neighbor, 'crossing')
        r2 = cost_cross + lower_crossing[neighbor]
        # print("cost:", cost_cross)
        # print("bound:", lower_crossing[neighbor])
        # print("R2:", r2)

    result.append(np.array([r1, r2])) # This is the new lower bound
    # print(result)

    for neighbor in G[current_node]:
        distance = np.sum(np.abs(t - result)) # Manhattan distance
        while distance < min_distance: # Pick the neighbor which has a result closest to the target
            min_distance = distance
            closest_node = neighbor
            neighbor = closest_node
        new_path = path + [neighbor]
        stack.append((neighbor, new_path))

        # Pruning paths that won't be Pareto-better compared to the current upper bound
        val_obj1_path = nx.path_weight(G, path=new_path, weight='length')
        val_obj2_path = nx.path_weight(G, path=new_path, weight='crossing')
        val_vector_path.append(np.array([val_obj1_path, val_obj2_path]))
        if any(val_vector_path[0]) > any(U):
            new_path = stack.pop()

    i += 1
    if max_iter is not None and i >= max_iter:
        print(f"Path {path} with cost {val_vector_path}")
        print("The algorithm has reached the given maximum iterations, but has found no solution.")
        break

    # print("The algorithm has terminated, but no solution was found.")
# S = (122245.37633330293, 486126.8581684635)
# T = (122246.77932030056, 486223.5791244763)

Path [(122245.37633330293, 486126.8581684635), (122254.86602688645, 486129.80052856216), (122264.35426393585, 486132.74411788705), (122273.84081337851, 486135.69101398275), (122283.19823912054, 486138.9678026403), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122284.1816391946, 486139.19731384714), (122290

In [ ]:
p_s = path
# Inner-loop approach
# import dfs_lower
# p_s = dfs_lower.dfs_lower(G, S, t, lower_bounds)
# print(p_s)

In [68]:
# If v^p_s improves in the target region
val_vector_p_s = []
val_p_s1 = nx.path_weight(G, path=p_s, weight='length')
val_p_s2 = nx.path_weight(G, path=p_s, weight='crossing')
val_vector_p_s.append(np.array([val_p_s1, val_p_s2]))
if val_p_s1 < U[0] and val_p_s2 < U[1]:

SyntaxError: incomplete input (799760698.py, line 4)

In [ ]:
P = P.append(p_s)

In [ ]:
#TODO: Test
#Compare p^s to p^∗ and add comparison to the GP ▷User ranking, i.e., is the new path preferred to the current, maximum one?
compare_ps_pstar = []
val_vector_p_star = []

val_p_star1 = nx.path_weight(G, path=p_star, weight='length')
val_p_star2 = nx.path_weight(G, path=p_star, weight='crossing')
val_vector_p_s.append(np.array([val_p_star1, val_p_star2]))

compare_ps_pstar.append(np.array([val_vector_p_s, val_vector_p_star]))

In [ ]:
ranking_new_paths = user_preference.get_preference(compare_ps_pstar, add_noise=add_noise)
print(ranking_new_paths)

In [ ]:
# Add the comparisons to GP
comparisons.add_single_comparison(compare_ps_pstar[np.argmax(ranking_new_paths)], compare_ps_pstar[
    np.argmin(ranking_new_paths)])  #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

In [ ]:
u_v_p_s, _ = gp.get_predictive_params(val_vector_p_s, True)  #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
u_v_p_star, _ = gp.get_predictive_params(val_vector_p_star, True)

In [ ]:
# if u(v^{p^s}) > u(v^{p^*}) then
if val_vector_p_s > val_vector_p_star:
    #p^∗ ← p^s
    p_star = p_s
# end if

In [ ]:
# Compute new candidate targets based on v^{p^s} and add to C
C = [min(val_vector_p_s[0][0], val_vector_p_s[1][0]), min(val_vector_p_s[0][1], val_vector_p_s[1][1])]
C

In [ ]:
# end if
# end while
# return p^∗, v^{p^*}
